# Цель задания: Исследование алгоритмов решения задач методом поиска. Описание предметной области. Имеется транспортная сеть, связывающая города СНГ. Сеть представлена в виде таблицы связей между городами. Связи являются двусторонними, т.е. допускают движение в обоих направлениях. Необходимо проложить маршрут из одной заданной точки в другую.
Этап 1. Неинформированный поиск. На этом этапе известна только топология связей между городами. Выполнить: 1)поиск в ширину; 2)поиск глубину; 3)поиск с ограничением глубины; 4)поиск с итеративным углублением; 5)двунаправленный поиск. Отобразить движение по дереву на его графе с указанием сложности каждого вида поиска. Сделать выводы.

In [366]:
import networkx as nx
from networkx import *
import pylab as plt
import os
import collections

In [367]:
table = '''Вильнюс Брест 531
Витебск Брест 638
Витебск Вильнюс 360
Воронеж Витебск 869
Воронеж Волгоград 581
Волгоград Витебск 1455
Витебск Ниж.Новгород 911
Вильнюс Даугавпилс 211
Калининград Брест 699
Калининград Вильнюс 333
Каунас Вильнюс 102
Киев Вильнюс 734
Киев Житомир 131
Житомир Донецк 863
Житомир Волгоград 1493
Кишинев Киев 467
Кишинев Донецк 812
С.Петербург Витебск 602
С.Петербург Калининград 739
С.Петербург Рига 641
Москва Казань 815
Москва Ниж.Новгород 411
Москва Минск 690
Москва Донецк 1084
Москва С.Петербург 664
Мурманск С.Петербург 1412
Мурманск Минск 2238
Орел Витебск 522
Орел Донецк 709
Орел Москва 368
Одесса Киев 487
Рига Каунас 267
Таллинн Рига 308
Харьков Киев 471
Харьков Симферополь 639
Ярославль Воронеж 739
Ярославль Минск 940
Уфа Казань 525
Уфа Самара 461'''
# изначальные данные
table

'Вильнюс Брест 531\nВитебск Брест 638\nВитебск Вильнюс 360\nВоронеж Витебск 869\nВоронеж Волгоград 581\nВолгоград Витебск 1455\nВитебск Ниж.Новгород 911\nВильнюс Даугавпилс 211\nКалининград Брест 699\nКалининград Вильнюс 333\nКаунас Вильнюс 102\nКиев Вильнюс 734\nКиев Житомир 131\nЖитомир Донецк 863\nЖитомир Волгоград 1493\nКишинев Киев 467\nКишинев Донецк 812\nС.Петербург Витебск 602\nС.Петербург Калининград 739\nС.Петербург Рига 641\nМосква Казань 815\nМосква Ниж.Новгород 411\nМосква Минск 690\nМосква Донецк 1084\nМосква С.Петербург 664\nМурманск С.Петербург 1412\nМурманск Минск 2238\nОрел Витебск 522\nОрел Донецк 709\nОрел Москва 368\nОдесса Киев 487\nРига Каунас 267\nТаллинн Рига 308\nХарьков Киев 471\nХарьков Симферополь 639\nЯрославль Воронеж 739\nЯрославль Минск 940\nУфа Казань 525\nУфа Самара 461'

In [368]:
graph = read_edgelist(r"data/data_city.txt") #считываем граф из файла, без веса
graph = nx.Graph(graph)
nx.draw(graph,  with_labels=True, pos=nx.circular_layout(graph))
plt.savefig('graph.png')
plt.close()
nx.draw(graph,  with_labels=True, pos=nx.kamada_kawai_layout(graph))
plt.savefig('graph_2.png')
plt.close()

In [369]:
table = list(map(lambda x: x[0:2] + [int(x[2])], map(lambda x: x.split(), table.split('\n'))))
#разбиваем на списки смежности данные с учетом веса
table

[['Вильнюс', 'Брест', 531],
 ['Витебск', 'Брест', 638],
 ['Витебск', 'Вильнюс', 360],
 ['Воронеж', 'Витебск', 869],
 ['Воронеж', 'Волгоград', 581],
 ['Волгоград', 'Витебск', 1455],
 ['Витебск', 'Ниж.Новгород', 911],
 ['Вильнюс', 'Даугавпилс', 211],
 ['Калининград', 'Брест', 699],
 ['Калининград', 'Вильнюс', 333],
 ['Каунас', 'Вильнюс', 102],
 ['Киев', 'Вильнюс', 734],
 ['Киев', 'Житомир', 131],
 ['Житомир', 'Донецк', 863],
 ['Житомир', 'Волгоград', 1493],
 ['Кишинев', 'Киев', 467],
 ['Кишинев', 'Донецк', 812],
 ['С.Петербург', 'Витебск', 602],
 ['С.Петербург', 'Калининград', 739],
 ['С.Петербург', 'Рига', 641],
 ['Москва', 'Казань', 815],
 ['Москва', 'Ниж.Новгород', 411],
 ['Москва', 'Минск', 690],
 ['Москва', 'Донецк', 1084],
 ['Москва', 'С.Петербург', 664],
 ['Мурманск', 'С.Петербург', 1412],
 ['Мурманск', 'Минск', 2238],
 ['Орел', 'Витебск', 522],
 ['Орел', 'Донецк', 709],
 ['Орел', 'Москва', 368],
 ['Одесса', 'Киев', 487],
 ['Рига', 'Каунас', 267],
 ['Таллинн', 'Рига', 308],
 ['Хар

In [370]:
#Мой вариант 7
START = 'Рига'
END = 'Одесса'
a_list = table
a_dict = {el[0]:dict() for el in a_list}
for el in a_list:
    a_dict[el[1]] = dict()
print(a_dict)
for el in a_list:
    a_dict[el[0]][el[1]] = el[2]
    a_dict[el[1]][el[0]] = el[2]
a_dict # вложенный словарь, с которым будем работать

{'Вильнюс': {}, 'Витебск': {}, 'Воронеж': {}, 'Волгоград': {}, 'Калининград': {}, 'Каунас': {}, 'Киев': {}, 'Житомир': {}, 'Кишинев': {}, 'С.Петербург': {}, 'Москва': {}, 'Мурманск': {}, 'Орел': {}, 'Одесса': {}, 'Рига': {}, 'Таллинн': {}, 'Харьков': {}, 'Ярославль': {}, 'Уфа': {}, 'Брест': {}, 'Ниж.Новгород': {}, 'Даугавпилс': {}, 'Донецк': {}, 'Казань': {}, 'Минск': {}, 'Симферополь': {}, 'Самара': {}}


{'Вильнюс': {'Брест': 531,
  'Витебск': 360,
  'Даугавпилс': 211,
  'Калининград': 333,
  'Каунас': 102,
  'Киев': 734},
 'Витебск': {'Брест': 638,
  'Вильнюс': 360,
  'Воронеж': 869,
  'Волгоград': 1455,
  'Ниж.Новгород': 911,
  'С.Петербург': 602,
  'Орел': 522},
 'Воронеж': {'Витебск': 869, 'Волгоград': 581, 'Ярославль': 739},
 'Волгоград': {'Воронеж': 581, 'Витебск': 1455, 'Житомир': 1493},
 'Калининград': {'Брест': 699, 'Вильнюс': 333, 'С.Петербург': 739},
 'Каунас': {'Вильнюс': 102, 'Рига': 267},
 'Киев': {'Вильнюс': 734,
  'Житомир': 131,
  'Кишинев': 467,
  'Одесса': 487,
  'Харьков': 471},
 'Житомир': {'Киев': 131, 'Донецк': 863, 'Волгоград': 1493},
 'Кишинев': {'Киев': 467, 'Донецк': 812},
 'С.Петербург': {'Витебск': 602,
  'Калининград': 739,
  'Рига': 641,
  'Москва': 664,
  'Мурманск': 1412},
 'Москва': {'Казань': 815,
  'Ниж.Новгород': 411,
  'Минск': 690,
  'Донецк': 1084,
  'С.Петербург': 664,
  'Орел': 368},
 'Мурманск': {'С.Петербург': 1412, 'Минск': 2238},
 'Орел': {

In [371]:
%%time
parents = dict()
parents[START]=None
class Queue:
    def __init__(self):
        self.elements = collections.deque()

    def empty(self):
        return len(self.elements) == 0

    def put(self, x):
        self.elements.append(x)

    def get(self):
        return self.elements.popleft()

def print_path():
    if END in parents: # если конечная вершина находится в списке с родителями, то выводим путь иначе решения нет
        path=[END]
        parent = parents[END]
        while not parent is None:
            path.append(parent)
            parent = parents[parent]
        print('LIMIT = ', len(path)-1)
        for x in  path[::-1]:
            print(x, ('---->' if x!=END else ' '), end=' ', )
    else:
        print('cant find')

CPU times: total: 0 ns
Wall time: 0 ns
Compiler : 156 ms


# Поиск в ширину
Трудоемкость O(V+E) рассматриваем все вершины и ребра единожды

In [372]:
%%time
def breadth_first_search_1(graph, start, end): #Трудоемкость O(V+E) рассматриваем все вершины и ребра единожды
    # печать того, что мы нашли
    frontier = Queue() # очередь
    frontier.put(start) # начинаем с вершины, которую передали в start
    visited = {} # пустой словарь вершин, которые посетили
    visited[start] = True
    global parents
    parents = {}
    parents[start] = None
    if start == end:
        return

    while not frontier.empty(): #пока очередь не пуста
        current = frontier.get() #берем элемент
     #   print("Visiting %r" % current) выводит обход графа в ширину
        for next in graph.neighbors(current): #смотрим всех соседей
            if next not in visited:
                frontier.put(next)
                parents[next] = current #записываем родителя для каждой вершины
                visited[next] = True

    path = [end]
    if end in parents:
        parent = parents[end]
        while not parent is None:
            path.append(parent)
            parent = parents[parent]
        for x in  path[::-1]:
            print(x, ('---->' if x!=end else ' поиск в ширину '), end=' ', )
    else:
        print('Нет пути между городами')



breadth_first_search_1(graph, START, END) # использую graph из библиотеки networkx

Рига ----> Каунас ----> Вильнюс ----> Киев ----> Одесса  поиск в ширину  CPU times: total: 0 ns
Wall time: 0 ns


In [373]:
%%time
parents = dict()
parents[START]=None
def dfs(graph, start, end, visited = None):  #function for dfs Трудоемкость O(V+E) рассматриваем все вершины и ребра единожды
    global parents
    if start == end:
        return
    visited = visited or set() # в python при or если одно из выражений True, то сразу принимает это значение
    visited.add(start)
    for neighbour in graph[start]:
        if neighbour not in visited:
            parents[neighbour] = start
            dfs(graph, neighbour, end, visited)


dfs(a_dict, START, END, {}) # здесь использую вложенный словарь, который создал в самом начале
print_path()

LIMIT =  6
Рига ----> С.Петербург ----> Витебск ----> Брест ----> Вильнюс ----> Киев ----> Одесса   CPU times: total: 0 ns
Wall time: 987 µs


# Поиск с ограничением глубины
Трудоемкость O(V+E) рассматриваем все вершины и ребра единожды

In [374]:
%%time
parents = dict()
parents[START]=None

def dls(graph, start, end, visited = None, limit = int, depth=0):  #function for dls Трудоемкость O(V+E) рассматриваем все вершины и ребра единожды
    visited = visited or list() # в python при or если одно из выражений True, то сразу принимает это значение
    global parents
    if depth > limit:
        return
    visited.append(start)
    for neighbour in graph[start]:
        if neighbour not in visited:
            parents[neighbour] = start
            dls(graph, neighbour, end, visited, limit, depth + 1)


dls(a_dict, START, END, [], 5) # здесь использую вложенный словарь, который создал в самом начале
print_path()

LIMIT =  6
Рига ----> С.Петербург ----> Витебск ----> Брест ----> Вильнюс ----> Киев ----> Одесса   CPU times: total: 0 ns
Wall time: 0 ns


# Поиск в глубину с итеративным углублением
Сумма времен поиска меньше, чем время при результате одного поиска от начало до цели O(V/2)+O(V/2) O(V)

In [375]:
%%time
parents = dict()
parents[START]=None

def dls(graph, start, end, visited = None, limit = int, depth=0):  #function for dls Трудоемкость O(V+E) рассматриваем все вершины и ребра единожды
    visited = visited or list() # в python при or если одно из выражений True, то сразу принимает это значение
    if depth > limit:
        return
    visited.append(start)
    for neighbour in graph[start]:
        if neighbour not in visited:
            parents[neighbour] = start
            dls(graph, neighbour, end, visited, limit, depth + 1)

LIMIT = 0
while END not in parents:
    LIMIT += 1
    dls(a_dict, START, END, [], LIMIT) # здесь использую вложенный словарь, который создал в самом начале
print_path()

LIMIT =  6
Рига ----> С.Петербург ----> Витебск ----> Брест ----> Вильнюс ----> Киев ----> Одесса   CPU times: total: 0 ns
Wall time: 998 µs


# Двунаправленный поиск

In [376]:
def check_in(list_1,list_2):
    a = set(list_1)
    b = set(list_2)
    if a&b:
        return True
    else:
        return False

In [377]:
%%time
def breadth_first_search_1(graph, start_1, start_2): #Трудоемкость O(V+E) рассматриваем все вершины и ребра единожды
    # печать того, что мы нашли

    frontier_1 = Queue() # очередь
    frontier_2 = Queue()
    frontier_1.put(start_1) # начинаем с вершины, которую передали в start
    frontier_2.put(start_2)
    visited_1 = {} # пустой словарь вершин, которые посетили
    visited_2 = {}
    visited_1[start_1] = True
    visited_2[start_2] = True
    parents_1 = {}
    parents_2 = {}
    parents_1[start_1] = None
    parents_2[start_2] = None

    while not check_in(visited_1, visited_2):
        if not frontier_1.empty(): #пока очередь не пуста
            current = frontier_1.get() #берем элемент
         #   print("Visiting %r" % current) выводит обход графа в ширину
            for next in graph.neighbors(current): #смотрим всех соседей
                if next not in visited_1:
                    frontier_1.put(next)
                    parents_1[next] = current #записываем родителя для каждой вершины
                    visited_1[next] = True
        if not frontier_2.empty(): #пока очередь не пуста
            current = frontier_2.get() #берем элемент
         #   print("Visiting %r" % current) выводит обход графа в ширину
            for next in graph.neighbors(current): #смотрим всех соседей
                if next not in visited_2:
                    frontier_2.put(next)
                    parents_2[next] = current #записываем родителя для каждой вершины
                    visited_2[next] = True
    global_city = list(set(visited_1)&set(visited_2)) # общий город

    # path_1 = [end]
    # parent_1 = parents[end]
    # while not parent is None:
    #     path.append(parent)
    #     parent = parents[parent]
    # for x in  path[::-1]:
    #     print(x, ('---->' if x!=end else ' поиск в ширину '), end=' ', )

    path_1 = [global_city[0]]
    parent_1 = parents_1[global_city[0]]
    while not parent_1 is None:
        path_1.append(parent_1)
        parent_1 = parents_1[parent_1]
    for x in  path_1[::-1]:
        print(x, ('-->' if x!=global_city[0] else '-->'), end=' ', )
    path_2 = [global_city[0]]
    parent_2 = parents_2[global_city[0]]
    while not parent_2 is None:
        path_2.append(parent_2)
        parent_2 = parents_2[parent_2]
    path_2.pop(0)
    for x in  path_2[:]:
        print(x, ('-->' if x!=path_2[-1] else ' двунаправленный поиск '), end=' ', )



breadth_first_search_1(graph, START, END) # использую graph из библиотеки networkx

Рига --> Каунас --> Вильнюс --> Киев --> Одесса  двунаправленный поиск  CPU times: total: 0 ns
Wall time: 0 ns


# Этап 2. Информированный поиск. Воспользовавшись информацией о
# протяженности связей от текущего узла, выполнить:
1) жадный поиск по первому наилучшему соответствию;
2) затем, использую информацию о расстоянии до цели по прямой от
каждого узла, выполнить поиск методом минимизации суммарной оценки
А*.
Отобразить на графе выбранный маршрут и сравнить его сложность с
неинформированным поиском. Сделать выводы.

In [378]:
def print_path_dist(finish):
    if finish in parents.keys(): # если конечная вершина находится в списке с родителями, то выводим путь иначе решения нет
        distance = 0
        path=[finish]
        # print(parents[finish])
        parent = list(parents[finish].keys())[0]
        while parents.get(parent) != 0:
            distance += parents[finish][parent]
            finish = parent
            path.append(parent)
            # print(parents.get(parent))
            parent = list(parents[parent].keys())[0]
        distance += parents[finish][parent]
        path.append(parent)
        print('LIMIT = ', len(path)-1)
        for x in  path[::-1]:
            print(x, ('---->' if x!=END else ' '), end=' ', )
        print('Пройденное расстояние', distance)
    else:
        print('cant find')

In [379]:
%%time
parents = dict()
def close_distance_dfs(graph, start, end, visited = None):  #
    global parents
    if start == end:
        return
    visited = visited or set() # в python при or если одно из выражений True, то сразу принимает это значение
    visited.add(start)
    for neighbour in sorted(graph[start], key = graph[start].get): #сортирует соседей по их близости
        if neighbour not in visited:
            parents[neighbour] = dict()
            parents[neighbour][start] = graph[neighbour][start] # записываю расстояние
            #parents[neighbour][start] = graph[neighbour][start]
            close_distance_dfs(graph, neighbour, end, visited)
parents[START]=0
close_distance_dfs(a_dict, START, END, {}) # здесь использую вложенный словарь, который создал в самом начале
print_path_dist(END)

LIMIT =  16
Рига ----> Каунас ----> Вильнюс ----> Калининград ----> Брест ----> Витебск ----> Орел ----> Москва ----> С.Петербург ----> Мурманск ----> Минск ----> Ярославль ----> Воронеж ----> Волгоград ----> Житомир ----> Киев ----> Одесса   Пройденное расстояние 11614
CPU times: total: 0 ns
Wall time: 998 µs


# Метод минимизации суммарной оценки А*
За эвристику возьмем расстояние с помощью bfs
Считаем расстояние до каждой вершины из Риги

In [380]:
distance = {}
def bfs_dist(graph, start, end): #Трудоемкость O(V+E) рассматриваем все вершины и ребра единожды
    # печать того, что мы нашли
    if start == end:
        return

    frontier = Queue() # очередь
    frontier.put(start) # начинаем с вершины, которую передали в start
    visited = {} # пустой словарь вершин, которые посетили
    visited[start] = True
    global parents
    parents = {}
    parents[start] = None
    global distance
    distance[start] = 0
    while not frontier.empty(): #пока очередь не пуста
        current = frontier.get() #берем элемент
        #print('current', current)
     #   print("Visiting %r" % current) выводит обход графа в ширину
        for next in graph[current]: #смотрим всех соседей
            if next not in visited:
                frontier.put(next)
                parents[next] = dict()
                prev_distance = distance[current]
                distance[next] = int(graph[next][current]) + int(prev_distance)
                parents[next][current] = graph[next][current]#записываем родителя для каждой вершины
                visited[next] = True
    print(distance)
    #print(len(distance))

    # path = [end]
    # parent = parents[end]
    # while not parent is None:
    #     path.append(parent)
    #     parent = parents[parent]
    # for x in  path[::-1]:
    #     print(x, ('---->' if x!=end else ' поиск в ширину '), end=' ', )



bfs_dist(a_dict, START, END) # использую graph из библиотеки networkx

{'Рига': 0, 'С.Петербург': 641, 'Каунас': 267, 'Таллинн': 308, 'Витебск': 1243, 'Калининград': 1380, 'Москва': 1305, 'Мурманск': 2053, 'Вильнюс': 369, 'Брест': 1881, 'Воронеж': 2112, 'Волгоград': 2698, 'Ниж.Новгород': 2154, 'Орел': 1765, 'Казань': 2120, 'Минск': 1995, 'Донецк': 2389, 'Даугавпилс': 580, 'Киев': 1103, 'Ярославль': 2851, 'Житомир': 4191, 'Уфа': 2645, 'Кишинев': 3201, 'Одесса': 1590, 'Харьков': 1574, 'Самара': 3106, 'Симферополь': 2213}


In [381]:
%%time
def A_star(graph, start, end, visited=None):
    global distance
    current = end
    d_dict = dict()
    path = list()
    path_dist = 0
    while current != start:
        for x in list(graph[current].keys()):
            d_dict[x] = distance[x]
        for neigh in sorted (d_dict, key = d_dict.get):
            print(d_dict)
            if neigh not in visited:
                print('current ', current, ' neigh', neigh)
                path_dist +=  int(graph[current][neigh])
                if distance[end] - distance[neigh] < path_dist:
                    visited.append(neigh)
                    path_dist = 0
                else:
                    path.append(current)
                    current = neigh
                        # print(current)
                    break

    path.append(start)
    for x in  path[::-1]:
        print(x, ('---->' if x!=END else ' '), end=' ', )
    print(path_dist)

A_star(a_dict, START, END, {})

{'Киев': 1103}
current  Одесса  neigh Киев
{'Киев': 1103, 'Вильнюс': 369, 'Житомир': 4191, 'Кишинев': 3201, 'Одесса': 1590, 'Харьков': 1574}
current  Киев  neigh Вильнюс
{'Киев': 1103, 'Вильнюс': 369, 'Житомир': 4191, 'Кишинев': 3201, 'Одесса': 1590, 'Харьков': 1574, 'Брест': 1881, 'Витебск': 1243, 'Даугавпилс': 580, 'Калининград': 1380, 'Каунас': 267}
current  Вильнюс  neigh Каунас
{'Киев': 1103, 'Вильнюс': 369, 'Житомир': 4191, 'Кишинев': 3201, 'Одесса': 1590, 'Харьков': 1574, 'Брест': 1881, 'Витебск': 1243, 'Даугавпилс': 580, 'Калининград': 1380, 'Каунас': 267, 'Рига': 0}
current  Каунас  neigh Рига
Рига ----> Каунас ----> Вильнюс ----> Киев ----> Одесса   1590
CPU times: total: 0 ns
Wall time: 0 ns


# Выводы
Метод минимизации суммарной оценки А* в данном случае показал оптимальный маршрут, так же как и двунаправленный поиск и поиск в ширину
Все неинформативные методы показали более лучший результат, чем метод жадного поиска 
